# Overview of BasicHullWhite

The [Hull-White model](https://en.wikipedia.org/wiki/Hull%E2%80%93White_model) is a short rate model represented by the stochastic differential equiation:

 $$dr(t) = (\theta(t) - a r(t))dt + \sigma dW$$


`BasicHullWhite` in the **economic** library is a simple implementation of the Hull-White model built using [modelx](https://github.com/fumitoh/modelx).

`BasicHullWhite` preforms Monte-Carlo simulations and generates paths of the instantaneous short rate based on the Hull-White model. It also inclues formulas to calculate various properties of the Hull-White model.

[Gouthaman Balaraman] presents some tests performed on a Hull-White model. He uses QuantLib to build his model, but the `BasicHullWhite` does not use QuantLib, and its Monte-Carlo simulations are generated from first principles using random numbers following the standard normal distribution. 
In addition, `BasicHullWhite` generates values of stochastic variable at each time step at once as a numpy array based on the vector modeling approach.

This notebook aims to perform analyses similar to Balaraman's using `BasicHullWhite`.  


[Gouthaman Balaraman]: http://gouthamanbalaraman.com/blog/hull-white-simulation-quantlib-python.html


<div class="alert alert-block alert-info">
    
**References**

* [Gouthaman Balaraman. Hull White Term Structure Simulations with QuantLib Python](http://gouthamanbalaraman.com/blog/hull-white-simulation-quantlib-python.html)
* [Gouthaman Balaraman. On the Convergence of Hull White Monte Carlo Simulations](http://gouthamanbalaraman.com/blog/hull-white-simulation-monte-carlo-convergence.html)    
* [Damiano Brigo, Fabio Mercurio (2006). Interest Rate Models - Theory and Practice with Smile, Inflation and Credit, 2nd ed.](https://link.springer.com/book/10.1007/978-3-540-34604-3)
* [Paul Glasserman (2003). Monte Carlo Methods in Financial Engineering](https://link.springer.com/book/10.1007/978-0-387-21617-1)


</div>




## Overview of the model

`HullWiteModel` include only one space, which is named `HullWhite`, and all the definitions are in that space. The `HullWhite` space is assined to `HW` in this notebook.

In [ ]:
import matplotlib.pyplot as plt
import modelx as mx

model = mx.read_model('BasicHullWhite')
HW = model.HullWhite

All the input parameters except for the initial curve are given as *References* (names starting with "_" are default names defined by modelx). 

In [ ]:
HW.refs

The defalut values for the number of scenarios, length of time ($T$), number of steps, $a$, $\sigma$ are set equal to the Balaraman's example.

In [ ]:
print("Number of scenarios:", HW.scen_size)
print("Length of time (in years):", HW.time_len)
print("Number of steps:", HW.step_size)
print("a:", HW.a)
print("sigma:", HW.sigma)

Below is a list of cells defined in `HW`. 

In [ ]:
HW.cells

Time-dependent functions are paremeterized with integer indexes instead of times themselves, 
i.e. $f(t)$ where $t=t_i, i=1, 2, 3, \ldots$ in math expression is translated as `f(i)`, `i=1, 2, 3...` in modelx formula.
Mapping $i$ to $t_i$ is done by `t_(i)`. By defalut, $t_i$s are evenly spaced, but the model should work even if the intervals are set uneven.

In [ ]:
HW.t_.formula

By default, the instanteneous forward rates observed at time 0 ($f^M(0, t_i)$) are set to 0.05 in `mkt_fwd` to be consistent with the Balaraman's example. $P^M(0, t_i)$, the market prices of zero-coupon bonds by duration are calculated from $f^M(0, t_i)$ in `mkt_zcb`. These may be defined the other way around, i.e. $f^M(0, t_i)$ may be derived from $P^M(0, t_i)$ inputs. The forward rates don't have to be constant.

In [ ]:
HW.mkt_fwd.formula

In [ ]:
HW.mkt_zcb.formula

`short_rate` corresponds to $r(t_{i})$, and recursively calculates stochastic paths of the instantaneous short rate at each time step.

In [ ]:
HW.short_rate.formula

Note that the initial stochastic differential equation, $dr(t) = (\theta(t) - a r(t))dt + \sigma dW$ is not used in this model.
Rather, the model uses the fact that the Hull-White model is a Gaussian process, 
and $r(t^i)$ conditional on $\mathcal{F}_{t_{i-1}} $ is normally distributed. `short_rate(i)` corresponds to the following expression

$$r(t_i) = E\{r(t_i) | \mathcal{F}_{t_{i-1}}\} + \sqrt{Var\{ r(t_i) | \mathcal{F}_{t_{i-1}} \}} * Z$$

where $Z$ represents random samples drawn from $\mathcal{N}(0, 1)$, the strandard normal distribution.

$E\{r(t_j) | \mathcal{F}_i\}$, the mean of $r(t_j)$ conditional on $\mathcal{F}_{t_i} $ is modeled as `E_rt_s(i, j)`. By replacing $t_{i}$ with $s$ and $t_{j}$ with $t$, the mean is expressed as:


$$ E\{r(t) | \mathcal{F}_{s}\} = r(s)e^{-a(t-s)}  + \alpha(t) - \alpha(s)e^{-a(t-s)} $$
   where 
   $$ \alpha(t) = f^M(0, t) + \frac{\sigma^2} {2a^2}(1-e^{-at})^2$$

In [ ]:
HW.E_rt_s.formula

In [ ]:
HW.E_rt.formula

In the same way, $Var\{r(t_{j}) | \mathcal{F}_{t_i}\}$, the variance of $r(t_j)$ conditional on $\mathcal{F}_{t_i}$ is modeled as `Var_rt_s(i, j)`. With the same definitions for $s$, $t$, $\alpha(t)$ as above, the variance is expressed as:

$$ Var\{ r(t) | \mathcal{F}_s \} = \frac{\sigma^2}{2a} (1 - e^{-2a(t-s)})$$


In [ ]:
HW.Var_rt_s.formula

Note that for each `i`, `short_rate(i)` returns a 1-dimensional numpy array having `scen_size` elements,
and for each pair of `i` and `j`, both `E_rt_s(i, j)` and `Var_rt_s(i, j)` also return an array with `scen_size` elements. 

$\alpha(t_{i})$ is modeled as `alpha(i)`. `alpha(i)` doesn't vary by scenario, so it returns a single value for each `i`.

In [ ]:
HW.alpha.formula

## Simulating $r(t_i)$

The chart below shows the first 10 paths of $r(t_i)$.
`short_rate_paths()` is defined to return $r(t_i)$ for all the scenarios and all the tim steps in a 2-dimensional array.

In [ ]:
HW.short_rate_paths.formula

In [ ]:
for i in range(10):
    plt.plot(range(HW.step_size+1), HW.short_rate_paths()[i])

For each $t_i$, the mean of $r(t_i)$ should converge to $E\{r(t_i) | \mathcal{F}_{0}\}$. For convenience, `mean_short_rate` and `E_rt` are defined to represent the mean of $r(t_i)$ and $E\{r(t_i) | \mathcal{F}_{0}\}$ respectively.

In [ ]:
HW.mean_short_rate.formula

In [ ]:
HW.E_rt.formula

The chart below compares the mean of $r(t_i)$ against $E\{r(t_i) | \mathcal{F}_{0}\}$ with 1000 scenarios. The chart looks similar to Balaraman's analysis. The mean converges pretty well during the first 20 years (240 steps), but diverges from the expectation around the 300th step.

In [ ]:
HW.scen_size = 1000
plt.plot(range(HW.step_size + 1), HW.E_rt(), "b-")
plt.plot(range(HW.step_size + 1), HW.mean_short_rate(), "r--")

The chart below is with 10,000 scenarios. The divergence dissapears and the mean fits the expectation much better.

In [ ]:
HW.scen_size = 10000
HW.a = 0.1
HW.sigma = 0.1
plt.plot(range(HW.step_size + 1), HW.E_rt(), "b-")
plt.plot(range(HW.step_size + 1), HW.mean_short_rate(), "r--")

In the same manner as the mean, for each $t_i$, the variance of $r(t_i)$ should converge to $Var\{r(t_i) | \mathcal{F}_{0}\}$. For convenience, `var_short_rate` and `Var_rt` are defined to represent the variance of $r(t_i)$ and $Var\{r(t_i) | \mathcal{F}_{0}\}$ respectively.

In [ ]:
HW.var_short_rate.formula

In [ ]:
HW.Var_rt.formula

In [ ]:
HW.scen_size = 1000
plt.plot(range(HW.step_size + 1), HW.Var_rt(), "b-")
plt.plot(range(HW.step_size + 1), HW.var_short_rate(), "r--")

In [ ]:
HW.scen_size = 10000
plt.plot(range(HW.step_size + 1), HW.Var_rt(), "b-")
plt.plot(range(HW.step_size + 1), HW.var_short_rate(), "r--")

## Simulating the discount factor

Along with $r(t_{i})$, the discount factor needs to be simulated. The discount factor is defined as $e^{-Y(t_i)}$ where

$$Y(t_i)=\int_0^{t_i}r(t)dt$$

For simplicity, we model $Y(t_i)$ as a descrete approximation to the integral:

$$\sum_{j=1}^{i}r(t_{j-1})(t_j-t_{j-1})$$

In [ ]:
HW.accum_short_rate.formula

There is an alternative approach to simulate $Y(t_i)$ by using the fact that $Y(t_i)$ follows a normal distribution, and by simulating the joint distribution of $(r(t_i), Y(t_i))$ as suggested in [Monte Carlo Methods in Financial Engineering](https://link.springer.com/book/10.1007/978-0-387-21617-1). `accum_short_rate2` implements this alternative approach, althogh it does not have material impact on the discussion below.  

In [ ]:
HW.accum_short_rate2.formula

`discount_factor` and `mean_disc_factor` are defined as follows.

In [ ]:
HW.disc_factor.formula

In [ ]:
HW.mean_disc_factor.formula

The chart below compares the mean of the simulated discount factors against $P^M(0, t_i)$ with 1000 scenarios. The mean diverges from the expectation significantly after the 150th step.

In [ ]:
HW.scen_size = 1000
plt.plot(range(HW.step_size+1), [HW.mkt_zcb(i) for i in range(HW.step_size+1)], "b-")
plt.plot(range(HW.step_size+1), HW.mean_disc_factor(), "r--")

The chart shows the case of 10,000 scenarios. The stochastic mean does not converge well. Even with 100,000 scenarios, the convergence is still poor.

In [ ]:
HW.scen_size = 10000
plt.plot(range(HW.step_size+1), [HW.mkt_zcb(i) for i in range(HW.step_size+1)], "b-")
plt.plot(range(HW.step_size+1), HW.mean_disc_factor(), "r--")

The charts below examine the convergence of the discount factor for various combinations of $\sigma$ and $a$, first by changing $\sigma$ and secondly by changing $a$.
As Balaraman's study shows, the convergence gets worse as $\sigma/a$ gets larger than 1, and gets better as $\sigma/a$ gets smaller than 1.

In [ ]:
HW.scen_size = 1000

fig, axs = plt.subplots(2, 2, sharex=True, sharey=True)
fig.suptitle(r"$a=$" + str(HW.a))
for sigma, (h, v) in zip([0.05, 0.075, 0.1, 0.125], [(0, 0), (0, 1), (1, 0), (1, 1)]):
    HW.sigma = sigma
    axs[h, v].set_title(r"$\sigma=$" + str(sigma) +  r", $\sigma/a=$" + "%.2f" % (sigma/HW.a))
    axs[h, v].plot(range(HW.step_size+1), [HW.mkt_zcb(i) for i in range(HW.step_size+1)], "b-")
    axs[h, v].plot(range(HW.step_size+1), HW.mean_disc_factor(), "r--")

In [ ]:
HW.scen_size = 1000
HW.sigma = 0.1

fig, axs = plt.subplots(2, 2, sharex=True, sharey=True)
fig.suptitle(r"$\sigma=$" + str(HW.sigma))
for a, (h, v) in zip([0.05, 0.1, 0.15, 0.2], [(0, 0), (0, 1), (1, 0), (1, 1)]):
    HW.a = a
    axs[h, v].set_title(r"$a=$" + str(a) +  r", $\sigma/a=$" + "%.2f" % (HW.sigma/HW.a))
    axs[h, v].plot(range(HW.step_size+1), [HW.mkt_zcb(i) for i in range(HW.step_size+1)], "b-")
    axs[h, v].plot(range(HW.step_size+1), HW.mean_disc_factor(), "r--")